In [ ]:
!conda install -c conda-forge spacy
!conda install -c conda-forge cupy
!pip install spacy_transformers

# Scegli uno tra:
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_trf

In [1]:
import json
    
# with open('dataset/SMS-NER-Dataset-110-Annotations/annotations.json', 'r') as file:
#    data = json.load(file)
    
with open('dataset/SMS-NER-Dataset-165-Annotations/annotations_new.json', 'r') as file:
     data = json.load(file)

In [2]:
data = data['annotations']
data = [tuple(i) for i in data]

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 41)

In [3]:
train_data[0]

('Paytm login detected from a new device at 03:14 PM, 06 May. Not you? To logout from all devices, click: https://ap.p y.tm/ys3MtK or report fraud @1800120130',
 {'entities': [[0, 5, 'TITLE'], [6, 20, 'PURPOSE'], [42, 59, 'TIME']]})

In [4]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_trf')
labels = nlp(train_data[0][0])
displacy.render(labels, style="ent", jupyter=True)

In [5]:
for i in train_data:
    if not i[1]['entities']:
        i[1]['entities'] = [(0, 0, 'PERSON')]
    else:
        for j in range(len(i[1]['entities'])):
            i[1]['entities'][j] = tuple(i[1]['entities'][j])

In [6]:
for i in test_data:
    if not i[1]['entities']:
        i[1]['entities'] = [(0, 0, 'PERSON')]
    else:
        for j in range(len(i[1]['entities'])):
            i[1]['entities'][j] = tuple(i[1]['entities'][j])

In [7]:
print(len(train_data), len(test_data))

132 33


In [8]:
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin


def make_doc_for_data(data):
    nlp = spacy.load("en_core_web_trf")

    db = DocBin() # create a DocBin object
    for text, annot in tqdm(data): # data in previous format
        if len(text) > 512:
            continue
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
        
    return db

make_doc_for_data(train_data).to_disk("train.spacy") # save the docbin object
make_doc_for_data(test_data).to_disk("test.spacy") # save the docbin object

100%|███████████████████████████████████████| 132/132 [00:00<00:00, 2385.18it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|█████████████████████████████████████████| 33/33 [00:00<00:00, 1664.71it/s]

Skipping entity
Skipping entity


In [9]:
!python -m spacy init fill-config dataset/SMS-NER-Dataset-165-Annotations/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
!python -m spacy train config.cfg --output ./output --paths.train train.spacy --paths.dev test.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  -------------  --------  ------  ------  ------  ------
  0       0        2946.75    910.18    0.20    0.11    1.15    0.00
 66     200      134052.61  64222.62   71.04   67.71   74.71    0.71
133     400        1100.12   1503.39   73.63   70.53   77.01    0.74
200     600         390.10    692.99   75.98   73.91   78.16    0.76
2

In [11]:
!python -m spacy benchmark accuracy model-165/trf/output/model-best model-165/test.spacy --output --code --gold-preproc --gpu-id 0 --displacy-path model-165/trf

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   73.91 
NER R   78.16 
NER F   75.98 
SPEED   483   


=============================== NER (per type) ===============================

               P        R        F
OTP        87.50    77.78    82.35
PURPOSE    60.00    65.22    62.50
TITLE      71.88    76.67    74.19
MONEY      75.00    81.82    78.26
TRANSAC   100.00   100.00   100.00
TIME       83.33   100.00    90.91

/home/musimathicslab/anaconda3/envs/cancer/lib/python3.9/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
✔ Generated 25 parses as HTML
/media/musimathicslab/My_Passport/estrazione-info-sms/trf
✔ Saved results to --code


In [349]:
# Download model-best for future use
import shutil
shutil.make_archive('model', 'zip', 'output')

'/home/musimathicslab/Desktop/Estrazione-Automatica-di-Informazioni-da-Testi/code/model.zip'